In [73]:
import os
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset
import json
from sklearn.model_selection import train_test_split
import lzma
import csv



In [74]:
#load data (unused)
"""dataset = pd.read_csv("../data/legal_text_classification.csv")
print(type(dataset))
print(dataset.columns)

dataset_text = dataset["case_text"]
print(dataset_text[0])
dataset_text = [f"summarize: {dtxt}" for dtxt in dataset_text]
print(dataset_text[0])

train_text, temp_text = train_test_split(dataset_text, test_size=0.2, random_state=1)
valid_text, test_text = train_test_split(temp_text, test_size=0.5, random_state=1)

with open("dataset/legal_train.json", "w") as f:
    json.dump({"input_data" : train_text}, f)
with open("dataset/legal_test.json", "w") as f:
    json.dump({"input_data" : test_text}, f)
with open("dataset/legal_valid.json", "w") as f:
    json.dump({"input_data" : valid_text}, f)"""

'dataset = pd.read_csv("../data/legal_text_classification.csv")\nprint(type(dataset))\nprint(dataset.columns)\n\ndataset_text = dataset["case_text"]\nprint(dataset_text[0])\ndataset_text = [f"summarize: {dtxt}" for dtxt in dataset_text]\nprint(dataset_text[0])\n\ntrain_text, temp_text = train_test_split(dataset_text, test_size=0.2, random_state=1)\nvalid_text, test_text = train_test_split(temp_text, test_size=0.5, random_state=1)\n\nwith open("dataset/legal_train.json", "w") as f:\n    json.dump({"input_data" : train_text}, f)\nwith open("dataset/legal_test.json", "w") as f:\n    json.dump({"input_data" : test_text}, f)\nwith open("dataset/legal_valid.json", "w") as f:\n    json.dump({"input_data" : valid_text}, f)'

In [75]:
#load data for json and convert it into a jsonl file
vol_one = "../data/TexasCaseDataVol1/json"
vol_two = "../data/TexasCaseDataVol2/json"
vol_three = "../data/TexasCaseDataVol3/json"
vol_four = "../data/TexasCaseDataVol4/json"
vol_five = "../data/TexasCaseDataVol5/json"
#vol_list = [vol_one, vol_two, vol_three, vol_four, vol_five]
vol_list = [vol_one, vol_two]
json_list = []
json_copy = None
count = 0
try:
    for v in vol_list:
        for f in os.listdir(v):
            path = os.path.join(v, f)
            with open(path, 'r') as file:
                json_data = {"article" : [], "highlights" : []}

                vol_json = json.load(file)
                json_copy = vol_json.copy()
                case_body = vol_json["casebody"]
                raw_text = case_body["opinions"][0]["text"]
                summary = case_body["head_matter"]
            
                
                json_data["article"] = "".join(raw_text).replace("\n", " ")
                json_data["highlights"] = "".join(summary).replace("\n", " ")

                json_list.append(json_data)
except Exception as e:
    print(f"Error with reading data from files: {e}")



"""try:
    with open("dataset/compiled_cases.jsonl", "w") as f:
        for d, s in zip(json_data["Document"], json_data["Summary"]):
            json.dump({"Document": d, "Summary": s}, f)
            f.write("\n")
except Exception as e:
    print(f"Error with saving json data: {e}")"""

'try:\n    with open("dataset/compiled_cases.jsonl", "w") as f:\n        for d, s in zip(json_data["Document"], json_data["Summary"]):\n            json.dump({"Document": d, "Summary": s}, f)\n            f.write("\n")\nexcept Exception as e:\n    print(f"Error with saving json data: {e}")'

In [76]:
#
json_df = pd.DataFrame(json_list)
json_df = json_df.dropna()

train_df, temp_df = train_test_split(json_df, test_size=0.4, random_state=1)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=1)

train_df.to_csv('dataset/train.csv', index=False)
test_df.to_csv('dataset/test.csv', index=False)
val_df.to_csv('dataset/validate.csv', index=False)